In [ ]:
import os
import numpy as np
import pandas as pd
import pre_processing_utils
from matplotlib.image import imread
from tensorflow.keras.metrics import AUC as auc
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = 'chest_xray'

In [ ]:
test_path = data_dir + '/test'
train_path = data_dir + '/train'
val_path = data_dir + '/val'

In [ ]:
image_shape = (300, 300, 3)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(fill_mode='nearest')

In [ ]:
image_gen.flow_from_directory(pre_processing_utils.train_path)

In [ ]:
image_gen.flow_from_directory(pre_processing_utils.test_path)

In [ ]:
image_gen.flow_from_directory(pre_processing_utils.validation_path)

<h2>Building a Model</h2>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, Layer, concatenate, GlobalAveragePooling2D,Activation, Softmax
from tensorflow.keras.metrics import Accuracy, AUC

In [ ]:
class fire_module(Layer):
    
    def __init__(self, squeeze_size=16, expand_size=64):
        super(fire_module, self).__init__()
        self.squeeze = Conv2D(filters=squeeze_size, kernel_size=(1,1), padding='valid', activation='relu', name=self.s_id + "sq1x1")
        self.exp1_1 = Conv2D(filters=expand_size, kernel_size=(1,1), padding='valid', activation='relu', name=self.s_id + "exp1x1")
        self.exp3_3 = Conv2D(filters=expand_size, kernel_size=(3,3), padding='same', activation='relu', name=self.s_id + "exp3x3")
    
    def call(self, input):
        squeezed_value = self.squeeze(input)
        exp1_1_value = self.exp1_1(squeezed_value)
        exp3_3_value = self.exp3_3(squeezed_value)
        return concatenate([exp1_1_value, exp3_3_value], axis=-1, name=self.s_id + 'concat')

In [ ]:
model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(3,3), input_shape=image_shape, activation='relu'))

model.add(fire_module(6,8))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(fire_module(12,16))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(fire_module(18,24))
model.add(MaxPool2D(pool_size=(3,3)))

model.add(Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid'))
model.add(GlobalAveragePooling2D())

#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=[AUC()])

In [ ]:
model.summary()

<h2>Regularization</h2>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import normalize

In [ ]:
early_stop = EarlyStopping(monitor='val_auc', mode='max', patience=2)
batch_size = 64

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size = image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode = 'binary')

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode='binary',
                                               shuffle=False) # Don't want to shuffle test data and lose labels

In [ ]:
test_image_gen.class_indices

<h3>Running the Model</h3>

In [ ]:
# Running without early stop for now - not enough examples for 'val_loss'?
results = model.fit(train_image_gen, epochs=20,
                             validation_data=test_image_gen,
                    callbacks=[early_stop]
                   )

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[['accuracy', 'val_accuracy']].plot()

In [ ]:
metrics[['auc_2', 'val_auc_2']].plot()

In [ ]:
metrics[['loss', 'val_loss']].plot()

<p>We see that the model is overfitting significantly.</p>

In [ ]:
model.save('image_classifier_accuracy.h5')